# Introduction to Convolutional Neural Networks


** Ecole Centrale Nantes **

** Diana Mateus **


** Participants : **



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py


### 0. Loading the dataset
Start by runing the following lines to load and visualize the data.

In [ ]:
# UNCOMMENT IF USING COLAB
from google.colab import drive
drive.mount('/content/drive')
IMDIR = '/content/drive/MyDrive/Colab Notebooks/'

In [ ]:
def load_dataset(IMDIR):
    train_dataset = h5py.File(IMDIR+'dataset/train_catvnoncat.h5', "r")
    train_x = np.array(train_dataset["train_set_x"][:])
    train_y = np.array(train_dataset["train_set_y"][:])
    test_dataset = h5py.File(IMDIR+'dataset/test_catvnoncat.h5', "r")
    test_x = np.array(test_dataset["test_set_x"][:])
    test_y = np.array(test_dataset["test_set_y"][:])
    classes = np.array(test_dataset["list_classes"][:])

    train_y = train_y.reshape((1, train_y.shape[0]))
    test_y = test_y.reshape((1, test_y.shape[0]))

    return train_x, train_y, test_x, test_y, classes

train_x, train_y, test_x, test_y, classes=load_dataset(IMDIR)

#### Visualize data

In [ ]:
# run several times to visualize different data points
# the title shows the ground truth class labels (0=no cat , 1 = cat)
index = np.random.randint(low=0,high=train_y.shape[1])
plt.imshow(train_x[index])
plt.title("Image "+str(index)+" label "+str(train_y[0,index]))
plt.show()
print ("Train X shape: " + str(train_x.shape))
print ("We have "+str(train_x.shape[0]),
       "images of dimensionality "
       + str(train_x.shape[1])+ "x"
       + str(train_x.shape[2])+ "x"
       + str(train_x.shape[3]))

### 1. CNNs with Keras and Tensorflow

Adapt the example in this website https://keras.io/examples/vision/mnist_convnet/ to our problem. To this end:
- change the number of classes and the input size
- remove the expand_dims(x_train, -1): it is not necessary to expand the dimensions since our input has already three channels
- you may need to transpose the labels vector
- change the categorical cross-entropy to the binary cross entropy given that our problem is binary classification.
- also change the softmax to sigmoid, the more appropriate activation function for binary data

We can choose a single neuron output passed through sigmoid, and then set a threshold to choose the class, or use two neuron output and then perform a softmax.

**2.2** Compute the train and test loss and accuracy after the model has been trained.  What model parameters does the ``fit`` function retain?

**2.3** How many parameters does the network have, explain  the exact number .

**2.4** Display and discuss the ROC curve of at least 3 different CNN configurations  


In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# the data, split between train and test sets
x_train, y_train, x_test, y_test, classes=load_dataset(IMDIR)

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# convert class vectors to binary class matrices
y_train = y_train.T
y_test = y_test.T

num_classes = 2
input_shape = (64, 64, 3)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



In [ ]:
#build the model
model = keras.Sequential(
    [
        keras.Input(shape= input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation = "relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="sigmoid")
    ]
)
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
#comiple and fit
batch_size = 32
epochs = 15

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

pred_original = model.predict(x_test)


In [ ]:
#evaluate
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

#2.1 The model is retaining all the weights and the biases

#2.2 It has 44482 parameters



In [ ]:
def wider_model():
  #create the model
  model_wider = keras.Sequential(
    [
        keras.Input(shape= input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(64, kernel_size=(5, 5), activation = "relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="sigmoid")
    ]
  )

  #comiple and fit
  batch_size = 32
  epochs = 15

  model_wider.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

  model_wider.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

  #evaluate
  score = model_wider.evaluate(x_test, y_test, verbose=0)
  print("Test loss:", score[0])
  print("Test accuracy:", score[1])
  y_pred = model_wider.predict(x_test)

  return y_pred



In [ ]:
pred_wider = wider_model()

In [ ]:
def deeper_model():
  #create the model
  model_deeper = keras.Sequential(
    [
        keras.Input(shape= input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Dropout(0.25),
        layers.Conv2D(64, kernel_size=(3, 3), activation = "relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Dropout(0.25),
        layers.Conv2D(128, kernel_size=(3, 3), activation = "relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Dropout(0.4),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="sigmoid")
    ]
  )

  #comiple and fit
  batch_size = 32
  epochs = 15

  model_deeper.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

  model_deeper.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

  #evaluate
  score = model_deeper.evaluate(x_test, y_test, verbose=0)
  print("Test loss:", score[0])
  print("Test accuracy:", score[1])
  y_pred = model_deeper.predict(x_test)
  plot_model(model_deeper, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
  return y_pred


In [ ]:
pred_deeper = deeper_model()

In [ ]:
def no_pooling_model():
  #create the model
  model_no_pooling = keras.Sequential(
    [
        keras.Input(shape= input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.Conv2D(64, kernel_size=(3, 3), activation = "relu"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="sigmoid")
    ]
  )

  #comiple and fit
  batch_size = 32
  epochs = 15

  model_no_pooling.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

  model_no_pooling.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)



  #evaluate
  score = model_no_pooling.evaluate(x_test, y_test, verbose=0)
  print("Test loss:", score[0])
  print("Test accuracy:", score[1])

  y_pred = model_no_pooling.predict(x_test)
  return y_pred

In [ ]:
pred_no_pooling = no_pooling_model()

In [ ]:
from sklearn import metrics

predictions = [pred_original, pred_wider, pred_deeper, pred_no_pooling]
names = ['Original', 'Wider', 'Deeper', 'No pooling']
i = 0
binary_labels = y_test[:, 1]
plt.figure(figsize=(10, 8))

for prediction in predictions:
  y_pred = prediction[:, 1]

  fpr, tpr, thresholds = metrics.roc_curve(binary_labels, y_pred)
  roc_auc = metrics.auc(fpr, tpr)

  # Plot ROC curve
  plt.plot(fpr, tpr, lw=2, label= names[i] + ' (area = {:.2f})'.format(roc_auc))
  i+= 1;

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()



## 2 Custom training loop (OPTIONAL ARTIN)
Replace the fit function by your own tensorflow  implementation

- Instantiate one of keras.optimizers to train the model.

- Instantiate a loss from keras.losses

- Define the metrics (from keras.metrics)

- Use `tf.data.Dataset.from_tensor_slices` to create an iterable dataset from a numpy arrays. Do this for the training and test datasets.

- Change the model (optional, after the training loop runs to optimize the performance)

- Program a loop over a fixed number of epochs,
    * For each epoch iterating over the batches
    * Within a `GradientTape()` scope,
      - do a forward pass on the model for the current batch (call the model on the batch data)
      - Compute the loss
      - Compute the gradients of the loss w.r.t parameters
      - Call the optimimzer to update the weights with computed the gradients
    * At the end of each epoch compute the validation metrics


Look at https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough specifically at the TRAINING LOOP SECTION
for a recent documentation on custom training.


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy

def manual_fit():
  model = keras.Sequential(
    [
        keras.Input(shape= input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation = "relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="sigmoid")
    ]
  )

  #comiple and fit
  batch_size = 32
  epochs = 15

  optimizer = Adam(learning_rate = 0.001)
  loss_fn = BinaryCrossentropy()
  accuracy_metric = BinaryAccuracy()

  train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
  test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

  for epoch in range(epochs):
    for batch_x, batch_y in train_dataset:
        with tf.GradientTape() as tape:
            # Forward pass
            logits = model(batch_x)

            loss_value = loss_fn(batch_y, logits)

        grads = tape.gradient(loss_value, model.trainable_variables)
        # Update weights
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        # Update accuracy metric
        accuracy_metric.update_state(batch_y, logits)

    # Compute validation metrics at the end of each epoch
    for batch_x_test, batch_y_test in test_dataset:
        logits_test = model(batch_x_test)
        accuracy_metric.update_state(batch_y_test, logits_test)

    print(f"Epoch {epoch + 1}: Loss = {loss_value:.4f}, Accuracy = {accuracy_metric.result().numpy():.4f}")
    accuracy_metric.reset_states()

In [ ]:
manual_fit()

ADITIONAL BONUS
- Early stopping
- Tensorboard
- CAM/GradCAM

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

def deeper_model_with_callbacks():
    model_deeper = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Dropout(0.25),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Dropout(0.25),
            layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Dropout(0.4),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation="sigmoid")
        ]
    )

    # Compile and fit with EarlyStopping and TensorBoard callbacks
    batch_size = 32
    epochs = 15

    model_deeper.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)


    model_deeper.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.1,
        callbacks=[early_stopping, tensorboard]
    )

    score = model_deeper.evaluate(x_test, y_test, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

In [ ]:
deeper_model_with_callbacks()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras

# Display
from IPython.display import Image, display
import matplotlib as mpl
import matplotlib.pyplot as plt

img_size = (64, 64)

# Assuming train_x[0] is your example image
img_array = train_x[156]
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

print(img_array.shape)
# Choose the last convolutional layer name
last_conv_layer_name = "conv2d_1"




In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:

img_array = preprocess_input(img_array)
print(img_array.shape)


# Make model
model = model

# Remove last layer's softmax
model.layers[-1].activation = None

# Print what the top predicted class is
# preds = model.predict(img_array)
# print("Predicted:", decode_predictions(preds, top=1)[0])

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()